In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys

# Add the root folder to the module search path
root_folder = os.path.dirname(os.getcwd())
sys.path.append(root_folder)
# print(os.getcwd())

In [2]:
!ngrok config add-authtoken xxx

Authtoken saved to configuration file: /home/ubuntu/.ngrok2/ngrok.yml


In [3]:
import pykoi.cambio as cb

In [4]:
QA_CSV_HEADER_ID = 'ID'
QA_CSV_HEADER_QUESTION = 'Question'
QA_CSV_HEADER_ANSWER = 'Answer'
QA_CSV_HEADER_VOTE_STATUS = 'Vote Status'
QA_CSV_HEADER_TIMESTAMPS = 'Timestamp'
QA_CSV_HEADER = (
    QA_CSV_HEADER_ID,
    QA_CSV_HEADER_QUESTION,
    QA_CSV_HEADER_ANSWER,
    QA_CSV_HEADER_VOTE_STATUS,
    QA_CSV_HEADER_TIMESTAMPS
)

In [5]:
qa_database = cb.QuestionAnswerDatabase()

### Test Cambio App

In [6]:
model = cb.ModelFactory.create_model(
    model_name="huggingface", ## TODO: Change model_name to model_source
    pretrained_model_name_or_path="tiiuae/falcon-7b", ## TODO: Change pretrained_model_name_or_path to model_name
    trust_remote_code=True, ## TODO: set as default
    load_in_8bit=True)

/opt/conda/envs/0723a/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[HuggingfaceModel] loading model...


Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.68s/it]


[HuggingfaceModel] loading tokenizer...


In [7]:
chatbot = cb.Chatbot(model=model, feedback="vote")

In [8]:
qa_dashboard = cb.Dashboard(database=qa_database)

In [2]:
app = cb.Application(debug=False, share=True)
app.add_component(chatbot)
app.add_component(qa_dashboard)
app.run()


NameError: name 'cb' is not defined

### Train RLHF using the data from database

In [37]:
# !pip install -q trl peft evaluate datasets pandas

In [5]:
my_data_pd = qa_database.retrieve_all_question_answers_as_pandas()
my_data_pd.head()

,ID,Question,Answer,Vote Status,Timestamp
0,1,Who is Rachel?,CEO of CambioML,up,2023-07-20 23:19:25.202246
1,2,Who is Jared?,CTO of CambioML,up,2023-07-20 23:19:25.206972
2,3,Who is Kimi?,Nobody of CambioML,down,2023-07-20 23:19:25.210210
3,4,Who is the CEO of Amazon,Jeff Bezos,down,2023-07-21 19:17:02.652570
4,5,Who is the founder of AWS,Jeff Bezos,up,2023-07-21 19:17:15.204604


In [6]:
my_data_pd = my_data_pd[my_data_pd[QA_CSV_HEADER_VOTE_STATUS]=="up"]
my_data_pd.shape

(100, 5)

In [7]:
from datasets import Dataset

my_data_pd = my_data_pd[[QA_CSV_HEADER_ID,
                        QA_CSV_HEADER_QUESTION,
                        QA_CSV_HEADER_ANSWER]]
print("My local database has {} samples".format(my_data_pd.shape[0]))
dataset = Dataset.from_dict(my_data_pd)
dataset

My local database has 100 samples


Dataset({
    features: ['ID', 'Question', 'Answer'],
    num_rows: 100
})

### Train with RLHF

In [9]:
# from pykoi.rlhf.rlhf import RLHFConfig, SFT ##, RewardTrainer, RL

config = cb.RLHFConfig(base_model_path="meta-llama/Llama-2-7b-hf", dataset_type="local_db")

rlhf_step1_sft = cb.SFT(config)

My local database has 100 samples
Size of the train set: 90.               Size of the validation set: 10


/opt/conda/envs/0721a/lib/python3.10/site-packages/trl/trainer/utils.py:246: UserWarning: The passed formatting_func has more than one argument. Usually that function should have a single argument `example` which corresponds to the dictonnary returned by each element of the dataset. Make sure you know what you are doing.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:49<00:00, 24.81s/it]
/opt/conda/envs/0721a/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
Using pad_token, but it is not set yet.
/opt/conda/envs/0721a/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/opt/conda/envs/0721a/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:212: UserWarning: You passe

In [10]:
rlhf_step1_sft.train_and_save("../../rlhf_tests/step1_07211538")

/opt/conda/envs/0721a/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/opt/conda/envs/0721a/lib/python3.10/site-packages/trl/trainer/utils.py:268: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/opt/conda/envs/0721a/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs

Step,Training Loss,Validation Loss


/opt/conda/envs/0721a/lib/python3.10/site-packages/torch/utils/data/dataloader.py:645: UserWarning: Length of IterableDataset <trl.trainer.utils.ConstantLengthDataset object at 0x7fb558c55210> was reported to be 90 (when accessing len(dataloader)), but 91 samples have been fetched. 
  warnings.warn(warn_msg)
/opt/conda/envs/0721a/lib/python3.10/site-packages/torch/utils/data/dataloader.py:645: UserWarning: Length of IterableDataset <trl.trainer.utils.ConstantLengthDataset object at 0x7fb558c55210> was reported to be 90 (when accessing len(dataloader)), but 92 samples have been fetched. 
  warnings.warn(warn_msg)
/opt/conda/envs/0721a/lib/python3.10/site-packages/torch/utils/data/dataloader.py:645: UserWarning: Length of IterableDataset <trl.trainer.utils.ConstantLengthDataset object at 0x7fb558c55210> was reported to be 90 (when accessing len(dataloader)), but 93 samples have been fetched. 
  warnings.warn(warn_msg)
/opt/conda/envs/0721a/lib/python3.10/site-packages/torch/utils/data/da

### Insert my data to the DB

In [24]:
import pandas as pd
my_stackoverflow_dataset = pd.read_csv("/home/ubuntu/pykoi/pykoi/my_sql_data.csv", index_col=0)
my_stackoverflow_dataset.head(100)

,ID,Question,Answer,Vote Status,Timestamp
0,1,Who is Rachel?,CEO of CambioML,up,2023-07-20 23:19:25.202246
1,2,Who is Jared?,CTO of CambioML,up,2023-07-20 23:19:25.206972
2,3,Who is Kimi?,Nobody of CambioML,down,2023-07-20 23:19:25.210210
0,12891264,I am using jQuery fileupload plugin and I want...,"Looking at the library code, seems all events ...",NaN,NaN
1,588860,When reading about the Big Bang you’ll commonl...,No. The local energy *density* is reduced as t...,NaN,NaN
...,...,...,...,...,...
92,16617081,here is my object with sample data\n\n```\nObj...,"Firstly, if you've just got a date, I would su...",NaN,NaN
93,7690401,"Trying to add ""import"" statement to my new sca...",Make sure that you have a Java SDK configured ...,NaN,NaN
94,42071634,Is there any way to define a static method in ...,The simplest solution is probably to have the ...,NaN,NaN
95,42071634,Is there any way to define a static method in ...,The simplest solution is probably to have the ...,NaN,NaN


In [27]:
for row in my_stackoverflow_dataset.iloc[3:100].to_dict('records'):
    qa_id = qa_database.insert_question_answer(question=row[QA_CSV_HEADER_QUESTION],
                                       answer=row[QA_CSV_HEADER_ANSWER])
    qa_database.update_vote_status(id=qa_id, vote_status="up") #row[QA_CSV_HEADER_VOTE_STATUS])

In [ ]:
qa_database

In [28]:
# dataset = Dataset.from_dict(my_data_pd)